In [1]:
import azureml.core

print("This notebook was created using version 1.0.23 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

This notebook was created using version 1.0.23 of the Azure ML SDK
You are currently using version 1.0.45 of the Azure ML SDK


In [2]:
import os
subscription_id = os.getenv("SUBSCRIPTION_ID", default="52f3cf55-fed4-4f7e-9aca-f3da535a03c1")
resource_group = os.getenv("RESOURCE_GROUP", default="automl_rg")
workspace_name = os.getenv("WORKSPACE_NAME", default="automl_ws")
workspace_region = os.getenv("WORKSPACE_REGION", default="eastus2")


In [3]:
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication

try:
    interactive_auth = InteractiveLoginAuthentication(tenant_id="1591aa04-9c2a-4144-9a04-fb8b0d506de5")
    ws = Workspace(subscription_id = subscription_id, 
               resource_group = resource_group, 
               workspace_name = workspace_name,
               auth=interactive_auth)    # write the details of the workspace to a configuration file to the notebook library
    ws.write_config()
    print("Workspace configuration succeeded.")
except:
    print("Workspace not accessible.")

Workspace configuration succeeded.


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print("Found existing cpucluster")
except ComputeTargetException:
    print("Creating new cpucluster")
    
    # Specify the configuration for the new cluster
    compute_config = AmlCompute.provisioning_configuration(vm_size="STANDARD_D2_V2",
                                                           min_nodes=0,
                                                           max_nodes=4)

    # Create the cluster with the specified name and configuration
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
    # Wait for the cluster to complete, show the output log
    cpu_cluster.wait_for_completion(show_output=True)

Found existing cpucluster


# Load data


In [33]:
import os
#dataset_name = "defect_camel_1_2"
#dataset_name = "defect_eclipse_2_0"
#dataset_name = "defect_eclipse_3_0"
#dataset_name = "defect_prop_2"
dataset_name = "defect_xalan_2_6"
dataset_file_name = dataset_name+'.csv'
dataset_path_name = os.path.join(os.getcwd(),'data/'+dataset_name+'.csv' )

In [34]:
import pandas as pd
df = pd.read_csv(dataset_path_name)
x_df = df.drop(columns =['target'])
y_df = df[['target']]
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size=0.2, random_state=42)
# flatten y_train to 1d array
y_train.values.flatten()# flatten y_train to 1d array


array([ True,  True, False,  True, False,  True, False, False, False,
        True, False,  True,  True,  True,  True, False, False, False,
        True,  True, False, False,  True, False, False, False,  True,
        True, False,  True,  True, False, False, False, False, False,
        True,  True,  True, False,  True,  True, False, False, False,
        True, False,  True, False,  True,  True, False, False,  True,
        True, False, False, False,  True,  True, False, False, False,
        True, False,  True,  True,  True,  True, False,  True,  True,
        True, False,  True,  True, False, False, False, False, False,
        True, False, False,  True, False,  True,  True,  True,  True,
       False, False,  True, False, False,  True, False,  True, False,
       False, False, False, False, False, False, False, False,  True,
        True,  True, False, False,  True,  True,  True, False, False,
        True,  True, False,  True, False,  True, False,  True, False,
        True,  True,

In [35]:
x_df.__len__()

885


# AutoML start

In [7]:
import azureml.core
import pandas as pd
from azureml.core.workspace import Workspace
import logging
import os

In [8]:
ws = Workspace.from_config()
# choose a name for the run history container in the workspace
experiment_name = 'Automl_'+dataset_name
# project folder
project_folder = './Automl/'+dataset_name

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

,
SDK version,1.0.45
Subscription ID,52f3cf55-fed4-4f7e-9aca-f3da535a03c1
Workspace,automl_ws
Resource Group,automl_rg
Location,canadacentral
Project Directory,./Automl/defect_xalan_2_6


In [9]:
automl_settings = {
    "iteration_timeout_minutes" : 60,
    "iterations" : 100,
    "primary_metric" : 'AUC_weighted',
    "preprocess" : True,
    "verbosity" : logging.INFO,
    "n_cross_validations": 100,
    "validation_size": 0.67,
    "enable_voting_ensemble":False,
    "enable_stack_ensemble":False,
    "model_explainability":True,
}

In [10]:
from azureml.train.automl import AutoMLConfig

# local compute 
automated_ml_config = AutoMLConfig(task = 'classification',
                             debug_log = 'automated_ml_errors_'+dataset_name+'.log',
                             path = project_folder,
                             X = x_train,
                             y = y_train.values.flatten(),
                             **automl_settings)

In [11]:
from azureml.core.experiment import Experiment
experiment = Experiment(ws, experiment_name)
remote_run = experiment.submit(automated_ml_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_561e7b85-1240-4bcc-aff3-5c674497ae0a
Current status: DatasetFeaturization. Beginning to featurize the dataset.
Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturizationCompleted. Completed featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION 

In [17]:
remote_run.status 

'Completed'

In [19]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 'sd…

In [20]:
from azureml.widgets import RunDetails
from azureml.train.automl.run import AutoMLRun
from azureml.core.experiment import Experiment
experiment=Experiment(ws, experiment_name)
experiment = Experiment(ws, 'Automl_defect_camel_1_2')

remote_run = AutoMLRun(experiment = experiment, run_id ="AutoML_29aecac8-d329-4ead-85e9-55db459752a2")

RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 'sd…

# Fetch best model

In [14]:
#iteration = 15
#b_run, b_model = remote_run.get_output(iteration=iteration)

b_run, b_model = remote_run.get_output()
print('Algorithm: '+b_run.get_properties()['run_algorithm'])
print(b_run)
print(b_model)

Algorithm: RandomForest
Run(Experiment: Automl_defect_xalan_2_6,
Id: AutoML_561e7b85-1240-4bcc-aff3-5c674497ae0a_80,
Type: None,
Status: Completed)
Pipeline(memory=None,
     steps=[('datatransformer', DataTransformer(enable_feature_sweeping=None, feature_sweeping_timeout=None,
        is_onnx_compatible=None, logger=None, observer=None, task=None)), ('MaxAbsScaler', MaxAbsScaler(copy=True)), ('RandomForestClassifier', RandomForestClassifier(bootstrap=True, class_weight=...imators=200, n_jobs=1,
            oob_score=True, random_state=None, verbose=0, warm_start=False))])
Y_transformer(['LabelEncoder', LabelEncoder()])


# Register model

In [15]:
modelname = 'AML'+dataset_name.split('defect_')[1]+'_'+b_run.get_properties()['run_algorithm']
b_run.register_model(model_name=modelname, model_path = 'outputs/model.pkl')

Model(workspace=Workspace.create(name='automl_ws', subscription_id='52f3cf55-fed4-4f7e-9aca-f3da535a03c1', resource_group='automl_rg'), name=AMLxalan_2_6_RandomForest, id=AMLxalan_2_6_RandomForest:1, version=1, tags={}, properties={})

In [16]:
from azureml.core import Workspace
from azureml.core.model import Model
ws = Workspace.from_config()
b_model=Model(ws, modelname)
model_path=model.download(target_dir=os.path.join(os.getcwd(),'aml_model/'+dataset_name.split('defect_')[1]+'/'+b_run.get_properties()['run_algorithm']), exist_ok=True)
import pickle
b_model = pickle.load(open(model_path, 'rb'))

NameError: name 'model' is not defined

# Evaluate the best model on test dataset

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(b_model).fit(x_test, y_test.values)
eli5.show_weights(perm)

In [ ]:
from sklearn.metrics import auc,accuracy_score,roc_auc_score,auc,roc_curve
from sklearn.utils import resample
sample_size = int(y_test.shape[0]*0.1)
boot_size = 100
auc_weighted_list=[]
for ite in range(1,boot_size):
    resample_x, resampel_y = resample(x_test,y_test
                                      #, n_samples=sample_size
                                      , replace=True, 
          random_state=ite)
    predicted_y = b_model.predict_proba(resample_x)
    auc_weighted=roc_auc_score(resampel_y,predicted_y[:,1],average='weighted')
    if auc_weighted < 0.5: 
        auc_weighted = 1-auc_weighted
    auc_weighted_list.append(auc_weighted)
    
    
import numpy as np
import matplotlib.pyplot as plt
from pylab import *
plt.rcParams["figure.figsize"] = (5, 3)
fig1, ax1 = plt.subplots()
figtitle = 'AML: distribution of XGBoost performance on Eclipse_2.0'
ax1.set_title(figtitle)
bp_dict = ax1.boxplot(auc_weighted_list,vert=False)
for line in bp_dict['medians']:
    # get position data for median line
    x, y = line.get_xydata()[1] # top of median line
    # overlay median value
    text(x, y, round(x,4),
         horizontalalignment='center') # draw above, centered


# Best Model 's explanation
Retrieve the explanation from the best_run. And explanation information includes:

1. shap_values: The explanation information generated by shap lib
2. expected_values: The expected value of the model applied to set of X_train data.
3. overall_summary: The model level feature importance values sorted in descending order
4. overall_imp: The feature names sorted in the same order as in overall_summary
5. per_class_summary: The class level feature importance values sorted in descending order. Only available for the classification case
6. per_class_imp: The feature names sorted in the same order as in per_class_summary. Only available for the classification case

Note:- The retrieve_model_explanation() API only works in case AutoML has been configured with 'model_explainability' flag set to True.

In [ ]:
from azureml.train.automl.automlexplainer import retrieve_model_explanation

shap_values, expected_values, overall_summary, overall_imp, per_class_summary, per_class_imp = \
    retrieve_model_explanation(b_run)

In [ ]:
print(overall_summary)
print(overall_imp)

In [ ]:
print(per_class_summary)
print(per_class_imp)

In [ ]:

from azureml.train.automl.automlexplainer import explain_model

shap_values, expected_values, overall_summary, overall_imp, per_class_summary, per_class_imp = \
    explain_model(b_model, x_train, x_test, features=x_train.columns)

In [ ]:
print(overall_summary)
print(overall_imp)